In [67]:
#dependencies

from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import os

import seaborn as sns
%matplotlib inline

# import datetime
# from dateutil.relativedelta import relativedelta
# from datetime import date

#import re
#import nltk
#from nltk.corpus import stopwords


In [68]:
# Read in csv's

temp_us = "..//practice_sources/US_youtube_trending_data.csv"
temp_br = "..//practice_sources/BR_youtube_trending_data.csv"
temp_ca = "..//practice_sources/CA_youtube_trending_data.csv"
temp_de = "..//practice_sources/DE_youtube_trending_data.csv"
temp_fr = "..//practice_sources/FR_youtube_trending_data.csv"
temp_gb = "..//practice_sources/GB_youtube_trending_data.csv"
temp_in = "..//practice_sources/IN_youtube_trending_data.csv"
temp_jp = "..//practice_sources/JP_youtube_trending_data.csv"
temp_kr = "..//practice_sources/KR_youtube_trending_data.csv"
temp_mx = "..//practice_sources/MX_youtube_trending_data.csv"
temp_ru = "..//practice_sources/RU_youtube_trending_data.csv"

us_df = pd.read_csv(temp_us)
br_df = pd.read_csv(temp_br)
ca_df = pd.read_csv(temp_ca)
de_df = pd.read_csv(temp_de)
fr_df = pd.read_csv(temp_fr)
gb_df = pd.read_csv(temp_gb)
in_df = pd.read_csv(temp_in)
jp_df = pd.read_csv(temp_jp)
kr_df = pd.read_csv(temp_kr)
mx_df = pd.read_csv(temp_mx)
ru_df = pd.read_csv(temp_ru)



In [69]:
# Updating categoryId column with corresponding string for all csv's

category_codes = {
    "1":'Film & Animation',
    "2":'Autos & Vehicles',
    "10":'Music',
    "15":'Pets & Animals',
    "17":'Sports',
    "18":'Short Movies',
    "19":'Travel & Events',
    "20":'Gaming',
    "21":'Videoblogging',
    "22":'People & Blogs',
    "23":'Comedy',
    "24":'Entertainment',
    "25":'News & Politics',
    "26":'Howto & Style',
    "27":'Education',
    "28":'Science & Technology',
    "29":'Nonprofits & Activism',
    "30":'Movies',
    "31":'Anime/Animation',
    "32":'Action/Adventure',
    "33":'Classics',
    "34":'Comedy',
    "35":'Documentary',
    "36":'Drama',
    "37":'Family',
    "38":'Foreign',
    "39":'Horror',
    "40":'Sci-Fi/Fantasy',
    "41":'Thriller',
    "42":'Shorts',
    "43":'Shows',
    "44":'Trailers'
}

cat_code = us_df["categoryId"].map(lambda x: category_codes[str(x)])
us_df['cat_codes'] = cat_code

cat_code = br_df["categoryId"].map(lambda x: category_codes[str(x)])
br_df['cat_codes'] = cat_code

cat_code = ca_df["categoryId"].map(lambda x: category_codes[str(x)])
ca_df['cat_codes'] = cat_code

cat_code = de_df["categoryId"].map(lambda x: category_codes[str(x)])
de_df['cat_codes'] = cat_code

cat_code = fr_df["categoryId"].map(lambda x: category_codes[str(x)])
fr_df['cat_codes'] = cat_code

cat_code = gb_df["categoryId"].map(lambda x: category_codes[str(x)])
gb_df['cat_codes'] = cat_code

cat_code = in_df["categoryId"].map(lambda x: category_codes[str(x)])
in_df['cat_codes'] = cat_code

cat_code = jp_df["categoryId"].map(lambda x: category_codes[str(x)])
jp_df['cat_codes'] = cat_code

cat_code = kr_df["categoryId"].map(lambda x: category_codes[str(x)])
kr_df['cat_codes'] = cat_code

cat_code = mx_df["categoryId"].map(lambda x: category_codes[str(x)])
mx_df['cat_codes'] = cat_code

cat_code = ru_df["categoryId"].map(lambda x: category_codes[str(x)])
ru_df['cat_codes'] = cat_code

In [70]:
# Add country string column

us_df['country'] = 'United States'
br_df['country'] = 'Brazil'
ca_df['country'] = 'Canada'
de_df['country'] = 'Germany'
fr_df['country'] = 'France'
gb_df['country'] = 'United Kingdom'
in_df['country'] = 'India'
jp_df['country'] = 'Japan'
kr_df['country'] = 'South Korea'
mx_df['country'] = 'Mexico'
ru_df['country'] = 'Russia'

In [71]:
#concat all 11 csv's

ind_dfs = [us_df, br_df, ca_df, de_df, fr_df, gb_df, in_df, jp_df, kr_df, mx_df, ru_df]
main_df = pd.concat(ind_dfs)

In [72]:
# remove unwanted pipes from tags with .split()

def split_tags(title):
    if title == '[none]': return []
    return title.lower().replace('"','').split('|')

main_df['title'] = main_df['title'].apply(split_tags)

In [73]:
# add trending days column

temp_df = main_df.groupby('video_id')['trending_date'].count().reset_index()
temp_df.rename({'trending_date':'trending_days'}, axis = 'columns', inplace = True)
main_df = main_df.merge(temp_df, on = 'video_id', how = 'inner').sort_values('trending_date')

In [74]:
main_df = main_df.drop_duplicates(subset = 'video_id', keep='last')

In [75]:
main_df['target'] = (main_df['trending_days'] > 8).astype(int)

In [76]:
############ dataframe for tags nlp model #######################

tags_df = main_df[['target', 'country', 'cat_codes', 'trending_days', 'tags']].reset_index()
tags_df

,index,target,country,cat_codes,trending_days,tags
0,977531,0,South Korea,Entertainment,1,[None]
1,977538,0,South Korea,Entertainment,1,이제만나러갑니다|이만갑|김정은|경호원|대남연락소|간첩|북한|탈북|탈북민
2,977568,0,South Korea,Science & Technology,1,갤럭시노트20|노트20|노트20울트라|갤럭시 노트20 울트라|Galaxy Note ...
3,977567,0,South Korea,Gaming,1,용느|만만|똘끼|박호두|주식|단타
4,977566,0,South Korea,Sports,1,MBC 스포츠플러스|엠스플뉴스|엠스플|스포츠|Sports|MLB|야구|야구는 엠스플...
...,...,...,...,...,...,...
241108,334957,0,United States,Gaming,6,among us|among us impostor|among us imposter|a...
241109,1086289,0,South Korea,Howto & Style,8,사나고|3D펜|3Dpen|만들기|making|3d프린터|3Dprinting
241110,334951,0,United States,Science & Technology,6,Apple Studio Display|studio display|apple disp...
241111,334945,1,Canada,Gaming,9,rocket league|rocket league pc|rocket league b...


In [40]:
tags_df.to_csv('..//MICA_P3/tags_df.csv')